### **FUNCTION CALLING**

- In API call, you can describe functions to these models: `gpt-3.5-turbo-0613` and `gpt-4-0613`.
  
- The model `intelligently` chooses to output a  `JSON` object containing arguments to call those functions.
  
- The model rather generates the `JSON` that you can used to call the function in your code.

### **APPLICATIONS OF FUNCTION CALLING**

- Create chatbots that answer questions by calling external API's (like chatgpt plugin).
    * email godataprof to check what days he posts new videos on youtube: `send_email(to: str, body: str)`
  
    * what is the current weather like in Dallas? : `get_current_weather(location: str, unit: 'celsius' | 'fahrenheit')`


- Convert natural language into API calls.
    * convert "Who are my top customers?" to `get_customers_by_revenue(start_date: str, end_date: str, limit: int)` and call your internal API.
  
- Extract structured data from text.
  
    * define a function called `extract_data(name: str, birthday: str)`, or sql_query(query: str)

In [1]:
import openai
import os
import textwrap
import json

In [2]:
openai.api_key = "sk-..."

In [3]:
completion = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[{"role": "user", "content": "What's the current weather in Dallas?"}],
)

reply_content = completion.choices[0].message.content

print(textwrap.fill(reply_content))


I cannot provide real-time information. Please check a reliable
weather website or app for the current weather in Dallas.


In [4]:
completion = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[{"role": "user", "content": "Send email to godataprof using theodondre@gmail.com"}],
)

reply_content = completion.choices[0].message.content

print(textwrap.fill(reply_content))

I'm sorry, but as an AI language model, I cannot access email services
or send emails on your behalf.


In [43]:
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)



def send_email(to: str, subject: str,  body: str):
    message = {
        "from": "theod@gmail.com",
        "to": to,
        "body": body,
        "subject": subject
    }
    return json.dumps(message)

In [16]:
GMAIL_ADDRESS="@gmail.com"
GMAIL_PASSWORD="..."
RECEIVER_EMAIL="@gmail.com"
SMTP="smtp.gmail.com"
PORT=587

In [44]:
from email.message import EmailMessage
import requests
import smtplib


def get_weather(api_key, city):
    url = "http://api.openweathermap.org/data/2.5/weather"
    params = {
        "q": city,
        "appid": api_key,
        "units": "imperial"
    }
    
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # Raise an exception for non-2xx responses
        data = response.json()
        
        # Extract relevant weather information
        temperature = data["main"]["temp"]
        weather_description = data["weather"][0]["description"]
        humidity = data["main"]["humidity"]
        
        # Return the weather data
        return {
            "temperature": temperature,
            "description": weather_description,
            "humidity": humidity
        }
    except requests.exceptions.RequestException as e:
        print("Error occurred during API request:", e)
        return None


def email_sender(subject, body):
    """send email"""
    try:
        sender_email = GMAIL_ADDRESS
        sender_password = GMAIL_PASSWORD
        receiver_email = RECEIVER_EMAIL

        msg = EmailMessage()
        msg.set_content(body)
        msg["Subject"] = subject
        msg["From"] = sender_email
        msg["To"] = receiver_email

        session = smtplib.SMTP(SMTP, PORT)
        session.starttls()
        session.login(sender_email, sender_password)
        session.send_message(msg, sender_email, receiver_email)
        session.quit()
        print("Mail Sent!")
    except Exception as e:
        print(f'Error sending email: {e}')



In [20]:
completion = openai.ChatCompletion.create(
    model="gpt-4-0613",
    messages=[{"role": "user", "content": "Send an email to godataprof to check when he's going to upload new video on youtube"}],
    functions=[
    {
        "name": "email_sender",
        "description": "send email to a youtuber",
        "parameters": {
            "type": "object",
            "properties": {
                "to": {
                    "type": "string",
                    "description": "email address of receiver, e.g. godataprof@gmail.com",
                    },
                "subject": {
                    "type": "string", 
                    "description": "subject of the email, e.g. Hello! GodataProf"
                },
                "body": {
                    "type": "string", 
                    "description": "content of the email, e.g. Checking to see what times you upload new youtube videos"
                },
            },
            "required": ["to", "subject", "body"],
        },
    }
    ],
function_call="auto",
)

In [24]:
reply_content = completion.choices[0].message
reply_content

<OpenAIObject at 0x7f804828f680> JSON: {
  "content": null,
  "function_call": {
    "arguments": "{\n  \"to\": \"godataprof@gmail.com\",\n  \"subject\": \"Upload Schedule Inquiry\",\n  \"body\": \"Hello godataprof,\n\n  I hope you're doing well.\n\n  I'm a big fan of your content on Youtube, particularly your work with Data Science. If possible, could you kindly let me know when you're planning to upload a new video again? \n\n  Looking forward to your upcoming content!\n\n  Best regards\"\n}",
    "name": "email_sender"
  },
  "role": "assistant"
}

In [36]:
funcs = reply_content['function_call']['arguments']

funcs = json.loads(funcs.replace("\n", ""))

print(funcs)

{'to': 'godataprof@gmail.com', 'subject': 'Upload Schedule Inquiry', 'body': "Hello godataprof,  I hope you're doing well.  I'm a big fan of your content on Youtube, particularly your work with Data Science. If possible, could you kindly let me know when you're planning to upload a new video again?   Looking forward to your upcoming content!  Best regards"}


In [37]:
type(funcs)

dict

In [38]:
email_sender(subject=funcs['subject'], body=funcs['body'])

Mail Sent!


In [57]:
location = "Accra"

completion = openai.ChatCompletion.create(
    model="gpt-4-0613",
    messages=[{"role": "user", "content": f"What's the weather like in {location}?"}],
    functions=[
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                    },
                "unit": {
                    "type": "string", 
                    "enum": ["celsius", "fahrenheit"]
                },
            },
            "required": ["location"],
        },
    }
    ],
function_call="auto",
)

In [58]:
reply_content = completion.choices[0].message
reply_content

<OpenAIObject at 0x7f80481d62c0> JSON: {
  "content": null,
  "function_call": {
    "arguments": "{\n  \"location\": \"Accra\"\n}",
    "name": "get_current_weather"
  },
  "role": "assistant"
}

In [59]:
reply_content = completion.choices[0].message

funcs = reply_content.to_dict()['function_call']['arguments']

funcs = json.loads(funcs)

print(funcs)

{'location': 'Accra'}


In [60]:
# Step 2, check if the model wants to call a function
if reply_content.get("function_call"):
    function_name = reply_content["function_call"]["name"]

    # Step 3, call the function
    # Note: the JSON response from the model may not be valid JSON
    function_response = get_current_weather(
            location=reply_content.get("location"),
            unit=reply_content.get("unit"),
    )

    # Step 4, send model the info on the function call and function response
    second_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=[
                {"role": "user", "content": f"What is the weather like in {location}?"},
                reply_content,
                {
                    "role": "function",
                    "name": function_name,
                    "content": function_response,
                },
            ],
        )

second_response

<OpenAIObject chat.completion id=chatcmpl-7RxSbZ592LlIpfb6ejcXByaBApWmM at 0x7f80481b6950> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "The weather in Accra is currently 72\u00b0F. It is sunny and windy.",
        "role": "assistant"
      }
    }
  ],
  "created": 1686897777,
  "id": "chatcmpl-7RxSbZ592LlIpfb6ejcXByaBApWmM",
  "model": "gpt-3.5-turbo-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 17,
    "prompt_tokens": 77,
    "total_tokens": 94
  }
}

In [61]:
api_key = "..."
city = funcs['location'].split(",")[0]

weather_data = get_weather(api_key, city)
if weather_data is not None:
    temperature = weather_data["temperature"]
    description = weather_data["description"]
    humidity = weather_data["humidity"]
    
    print(f"Weather in {funcs['location']}:")
    print(f"Temperature: {temperature}°F")
    print(f"Description: {description}")
    print(f"Humidity: {humidity}%")


Weather in Accra:
Temperature: 77.14°F
Description: overcast clouds
Humidity: 79%
